In [ ]:
# Import library yang dibutuhkan
import os
import sys
from getpass import getpass  # Untuk input password tersembunyi

# Import komponen dari library cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

In [ ]:
# Tentukan konstanta sesuai desain
UKURAN_BLOK_AES_BYTES = 16  # 128 bits
UKURAN_KUNCI_BYTES = 32     # 256 bits
UKURAN_SALT_BYTES = 16
UKURAN_FILE_MAKS_BYTES = 1_048_576 # 1 MB (sesuai KAK)

In [ ]:
# Backend untuk operasi kriptografi
backend = default_backend()

In [ ]:
def derive_key(password: str, salt: bytes) -> bytes:
    """
    Mengubah password (string) menjadi kunci 256-bit (bytes)
    menggunakan PBKDF2.
    """
    print("Mendapatkan kunci dari password...")
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=UKURAN_KUNCI_BYTES,
        salt=salt,
        iterations=100000, # Iterasi standar untuk keamanan
        backend=backend
    )
    return kdf.derive(password.encode('utf-8'))

In [ ]:
def encrypt_file(password: str, file_input: str, file_output: str):
    """
    Fungsi untuk mengenkripsi file.
    Format file output: [ SALT | IV | CIPHERTEXT ]
    """
    try:
        # 1. Baca file input
        with open(file_input, 'rb') as f:
            plaintext = f.read()
    except FileNotFoundError:
        print(f"Error: File input '{file_input}' tidak ditemukan.")
        return
    except Exception as e:
        print(f"Error saat membaca file: {e}")
        return

    # 2. Cek batasan ukuran file (sesuai KAK)
    if len(plaintext) > UKURAN_FILE_MAKS_BYTES:
        print(f"Error: Ukuran file melebihi 1 MB. Ukuran: {len(plaintext)} bytes.")
        return

    # 3. Buat Salt dan Kunci
    salt = os.urandom(UKURAN_SALT_BYTES)
    key = derive_key(password, salt)

    # 4. Padding data (PKCS#7)
    padder = PKCS7(algorithms.AES.block_size).padder()
    data_padded = padder.update(plaintext) + padder.finalize()

    # 5. Buat IV (Initialization Vector)
    iv = os.urandom(UKURAN_BLOK_AES_BYTES)

    # 6. Inisialisasi Cipher AES-256 mode CBC
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    encryptor = cipher.encryptor()

    # 7. Enkripsi data
    print("Mulai mengenkripsi file...")
    ciphertext = encryptor.update(data_padded) + encryptor.finalize()

    # 8. Tulis ke file output (Salt + IV + Ciphertext)
    try:
        with open(file_output, 'wb') as f:
            f.write(salt)
            f.write(iv)
            f.write(ciphertext)
        print(f"✅ File berhasil dienkripsi ke: {file_output}")
    except Exception as e:
        print(f"Error saat menulis file output: {e}")

In [ ]:
def decrypt_file(password: str, file_input: str, file_output: str):
    """
    Fungsi untuk mendekripsi file.
    Membaca format file: [ SALT | IV | CIPHERTEXT ]
    """
    try:
        # 1. Baca file terenkripsi
        with open(file_input, 'rb') as f:
            data_terenkripsi = f.read()
    except FileNotFoundError:
        print(f"Error: File input '{file_input}' tidak ditemukan.")
        return
    except Exception as e:
        print(f"Error saat membaca file: {e}")
        return

    try:
        # 2. Ekstrak Salt, IV, dan Ciphertext
        salt = data_terenkripsi[:UKURAN_SALT_BYTES]
        iv = data_terenkripsi[UKURAN_SALT_BYTES : UKURAN_SALT_BYTES + UKURAN_BLOK_AES_BYTES]
        ciphertext = data_terenkripsi[UKURAN_SALT_BYTES + UKURAN_BLOK_AES_BYTES:]

        if len(ciphertext) == 0:
            print("Error: File terenkripsi tidak valid atau kosong.")
            return

        # 3. Buat Kunci dari Password dan Salt
        key = derive_key(password, salt)

        # 4. Inisialisasi Cipher AES-256 mode CBC
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
        decryptor = cipher.decryptor()

        # 5. Dekripsi data
        print("Mulai mendekripsi file...")
        data_padded = decryptor.update(ciphertext) + decryptor.finalize()

        # 6. Hapus Padding (PKCS#7)
        unpadder = PKCS7(algorithms.AES.block_size).unpadder()
        data_original = unpadder.update(data_padded) + unpadder.finalize()

    except Exception as e:
        # Ini PENTING: Jika password salah, proses unpadding akan gagal.
        print("\n❌ GAGAL DEKRIPSI!")
        print("Kemungkinan besar password Anda salah atau file korup.")
        # print(f"Detail error (debug): {e}") # Aktifkan untuk debug
        return

    # 7. Tulis ke file output
    try:
        with open(file_output, 'wb') as f:
            f.write(data_original)
        print(f"✅ File berhasil didekripsi ke: {file_output}")
    except Exception as e:
        print(f"Error saat menulis file output: {e}")

In [15]:
def main():
    """
    Fungsi utama untuk menjalankan aplikasi CLI.
    """
    print("========================================")
    print("  Aplikasi Kriptografi AES-256 (CBC)    ")
    print("      Nama: Muhammad Fikri Hanif        ")
    print("          NIM: 103052300118             ")
    print("           Kelas: DS-47-03              ")
    print("      Mata Kuliah: Keamanan Data        ")
    print("========================================")

    while True:
        mode = input("\nPilih mode (1: Enkripsi, 2: Dekripsi, 0: Keluar): ")

        if mode == '0':
            print("Terima kasih, program selesai.")
            break

        if mode not in ('1', '2'):
            print("Pilihan tidak valid, silakan coba lagi.")
            continue

        file_input = input("Masukkan nama file input: ")
        file_output = input("Masukkan nama file output: ")

        # Gunakan getpass agar password tidak terlihat saat diketik
        password = getpass("Masukkan password: ")
        if not password:
            print("Password tidak boleh kosong.")
            continue

        if mode == '1':
            print(f"\nMode: Enkripsi")
            print(f"Input: {file_input}")
            print(f"Output: {file_output}")
            encrypt_file(password, file_input, file_output)
        elif mode == '2':
            print(f"\nMode: Dekripsi")
            print(f"Input: {file_input}")
            print(f"Output: {file_output}")
            decrypt_file(password, file_input, file_output)

In [ ]:
main()